# Bag of Words-Modelle für Regionalzeitungen, gegliedert nach Bundesländern

## Benötigte Libraries

In [1]:
# loading libs
import pandas as pd
import numpy as np
import pprint as pp
import Stemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer

## Laden der Datensätze

In [2]:
bayern = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/bayern_demojized.csv", index_col=0)
bawü = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/bawü_demojized.csv", index_col=0)
berlin = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/berlin_demojized.csv", index_col=0)
bremen = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/bremen_demojized.csv", index_col=0)
hamburg = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/hamburg_demojized.csv", index_col=0)
hessen = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/hessen_demojized.csv", index_col=0)
brandenburg = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/brandenburg_demojized.csv", index_col=0)
mecklenburg = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/meckPomm_demojized.csv", index_col=0)
niedersachsen = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/niedersachsen_demojized.csv", index_col=0)
nrw = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/nrw_demojized.csv", index_col=0)
rlp = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/rlp_demojized.csv", index_col=0)
saarland = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/saarland_demojized.csv", index_col=0)
sachsen = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/sachsen_demojized.csv", index_col=0)
sachsenAnhalt = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/sachsen_anhalt_demojized.csv", index_col=0)
schleswig = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/schleswig_demojized.csv", index_col=0)
thüringen = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/thüringen_demojized.csv", index_col=0)

/tmp/ipykernel_44972/2606450382.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  bayern = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/bayern_demojized.csv", index_col=0)
/tmp/ipykernel_44972/2606450382.py:5: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  hamburg = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/hamburg_demojized.csv", index_col=0)
/tmp/ipykernel_44972/2606450382.py:9: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  niedersachsen = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/niedersachsen_demojized.csv", index_col=0)


## ÜBERGREIFENDE THEMEN FÜR ALLE LÄNDER

*Hier findet sich zuerst die Themenzusammenstellung wieder, welche bereits in einem gesonderten Dokument erklärt worden war. Dabei handelt es sich um die wichtigsten sowie immer wieder auftauchenden Themen in einem Querschnitt über alle einzeln analysierten Wörter-Bags hinweg. Dazu wurden dann auch eigene Begriffe mit in die Themen aufgenommen.*

In [3]:
## version: 24/10/2023

themenliste = {
    'protesteIran' : ['iran', 'iraner', 'iranerin', 'iranerinnen', 'iranisch', 'iranische', 'iranischen',
                      'mullah', 'mullahs', 'mullah-regime', 'theokratie', 'scharia', 'jin', 'jiyan', 'azadi',
                      'jina', 'mahsa', 'amini', 'mahsaamini', 'kopftuch', 'hijab', 'hidschab', 'jinjiyanazadi',
                      'frauenlebenfreiheit', 'womenlifefreedom', 'internetblockade', 'freeiran',
                      'iranprotests', 'iranprotests2022', 'iranrevolution', 'iranrevolution2022',
                      'sittenwächter', 'sittenpolizei', 'sittenpolizist', 'sittenpolizisten',
                      'kopftuchprotest', 'kopftuchproteste', 'hedschab', 'kopftücher', 'ajatollah', 'ayatollah',
                      'chamenei', 'khamenei'],
    
    'klimaKlimawandel' : ['klima', 'klimakrise', 'erwärmung', 'erderwärmung', 'co2', 'ökosystem', 'ökosysteme',
                          'kipppunkt', 'kipppunkte', 'artensterben', 'klimawandel', 'kohlendioxid',
                          'ökologisch', 'ökologische', 'ökologischer', 'ökosystem', 'emission', 'emissionen'],
    
    'klimaAktivismus' : ['lützerath', 'climateaction', 'fridaysforfuture', 'fff', 'greta', 'thunberg',
                         'letztegeneration', 'klimakleber', 'klimaproteste', 'klimaprotest', 'sitzblockade',
                         'sitzblockaden', 'klimaaktivist', 'klimaaktivistin', 'klimaaktivisten',
                         'klimaaktivistinnen', 'klimaaktivismus', 'klimademo', 'klimademonstration',
                         'klima-demo', 'klima-demonstration', 'klimademonstrant', 'klimademonstranten',
                         'klima-demonstrant', 'klima-demonstranten', 'greenpeace', 'sitzstreik', 'sitzstreiks',
                         'festkleben', 'festgeklebt'],
    
    'klimaPolitisch' : ['klimaschutz', 'energiewende', 'klimaschutzgesetz', 'kohleausstieg', 'ausbau',
                        'klimakanzler', 'dekarbonisierung', 'entkarbonisierung', 'umweltschutz', 'atomausstieg',
                        'emissionshandel', 'klimakonferenz', 'klimaabkommen', 'cop27', 'kyoto',
                        'kyoto-protokoll', 'klimaneutralität', 'klimaneutral', 'co2-neutralität', 'co2-neutral',
                        'co2neutralität', 'co2neutral', 'klimanotstand', 'naturschutz', 'klimapolitik',
                        'ölausstieg'],
    
    'klimaTechnologien' : ['solar', 'solarenergie', 'solartechnik', 'photovoltaik', 'fotovoltaik',
                           'photovoltaikanlage', 'fotovoltaikanlage', 'photovoltaikanlagen',
                           'fotovoltaikanlagen', 'photovoltaik-anlage', 'fotovoltaik-anlage',
                           'photovoltaik-anlagen', 'fotovoltaik-anlagen', 'geothermie', 'wasserkraft',
                           'wasserkraftwerk', 'wasserkraftwerke', 'erneuerbare', 'erneuerbaren', 'wind',
                           'windkraft', 'windkraftwerk', 'ausbau', 'nachhaltig', 'nachhaltige', 'nachhaltiger',
                           'nachhaltigkeit', 'ökostrom', 'renewables', 'renewable', 'solarthermie',
                           'erneuerbareenergien', 'erneuerbarenenergien'],
    
#     'klima' : ['klima', 'klimaschutz', 'klimakrise', 'energiewende', 'erwärmung', 'erderwärmung',
#                'klimaschutzgesetz', 'solar', 'solarenergie', 'photovoltaik', 'wasserkraft', 'kohleausstieg',
#                'erneuerbare', 'erneuerbaren', 'wind', 'windkraft', 'windkraftwerk', 'lützerath', 'ausbau',
#                'climateaction', 'fridaysforfuture', 'fff', 'greta', 'thunberg', 'klimakanzler', 'nachhaltig',
#                'nachhaltige', 'nachhaltiger', 'nachhaltigkeit', 'letztegeneration',' klimakleber',
#                'klimaprotest', 'klimaproteste', 'co2', 'sitzblockade', 'sitzblockaden', 'ökosystem',
#                'ökosysteme', 'kipppunkt', 'kipppunkte', 'solartechnik', 'photovoltaikanlage',
#                'photovoltaikanlagen', 'photovoltaik-anlage', 'photovoltaik-anlagen', 'klimaaktivist',
#                'klimaaktivistin', 'klimaaktivisten', 'klimaaktivistinnen', 'klimaaktivismus', 'klimademo',
#                'klimademonstration', 'klima-demo', 'klima-demonstration', 'klimademonstrant',
#                'klimademonstranten', 'klima-demonstrant', 'klima-demonstranten', 'dekarbonisierung',
#                'entkarbonisierung', 'emission', 'emissionen', 'umweltschutz', 'sitzstreik', 'sitzstreiks',
#                'enneuerbareenergien', 'erneuerbarenenergien', 'atomausstieg', 'kohleausstieg', 'artensterben',
#                'emissionshandel', 'gaskraft', 'gaskraftwerk', 'gaskraftwerke', 'greta', 'thunberg',
#                'greenpeace', 'klimakonferenz', 'cop27', 'kyoto', 'klimaneutralität', 'klimaneutral',
#                'co2-neutral', 'co2neutral', 'klimanotstand', 'klimawandel', 'kohlendioxid', 'naturschutz',
#                'klimapolitik', 'ökostrom', 'ökologisch', 'ökologische', 'ökologischer', 'ökosystem',
#                'ölausstieg', 'renewable', 'renewables', 'solarthermie'],
    
    'politikEuropa' : ['eu', 'europa', 'europäische', 'europäischen', 'europäischer', 'europäisches', 'european',
                       'europeanunion', 'europe', 'kommission', 'europarat', 'brexit', 'europawahl',
                       'ewr', 'eurozone', 'ezb', 'eugh', 'schengen', 'europapolitik', 'ministerrat'],
    
    'politikInternational' : ['außenpolitik', 'außenministerin', 'baerbock', 'diplomatie', 'usa', 'china', 'un',
                              'uno', 'nato', 'vereinte', 'nationen', 'unitednations', 'international',
                              'internationale', 'entwicklungshilfe'],
    
    'politikNational' : ['ampel', 'ampelkoalition', 'verticaltrafficlight', 'koalitionsvertrag', 'koalition',
                         'bundesregierung', 'regierung', 'minister', 'mehrheit', 'bundestag', 'ausschuss',
                         'gesetzentwurf', 'abstimmung', 'ausschüsse', 'bundespräsident', 'bundeskanzler',
                         'kanzler', 'länder', 'ländern', 'ostdeutschland', 'opposition', 'debatte', 'wahl',
                         'wahlkampf', 'gewählt', 'wählen', 'wähler', 'wählerin', 'wählerinnen', 'partei',
                         'fraktion', 'spd', 'gruene', 'gruenen', 'grüne', 'grünen', 'fdp', 'union', 'cdu',
                         'csu', 'afd', 'linke', 'linken', 'dielinke', 'ssw',
                         'ostdeutsche', 'ostdeutscher', 'ostdeutsch', 'westdeutsch', 'grün', 'gruen',
                         'westdeutschland', 'westdeutsche', 'westdeutscher', 'bureg', 'diegrünen', 'diegruenen'],
    
    'verteidigungspolitik' : ['zeitenwende', 'modernisieren', 'modernisierung', 'verteidigungspolitik',
                              'verteidigungsministerin', 'verteidigungsminister', 'verteidigungsministerium',
                              'sondervermögen', 'bundeswehr', 'truppe', 'heer', 'lambrecht', 'pistorius',
                              'verteidigungsfähig', 'verteidigungsfähigkeit', 'landesverteidigung'],
    
    'flutAhrtal' : ['flut', 'flutkatastrophe', 'ahrtal', 'ahr', 'neuenahr', 'ahrweiler', 'lewentz', 'hochwasser'],
    
    'polizistenmordKusel' : ['polizistenmord', 'polizistenmorde', 'polizistenmorden', 'kusel',
                             'mayweilerhof', 'ulmet'],
    
    'sozialesWohnen' : ['wohnungsbau', 'wohnen', 'bauen', 'mieter', 'bauen', 'mieterinnen', 'mieterin', 'miete',
                        'mietendeckel', 'mietenstopp', 'sozialwohnung', 'sozialwohnungen'],
    
    'sozialesEntlastungen' : ['inflation', 'sorgen', 'preise', 'energiepreise', 'lebenshaltungskosten',
                              'lebenshaltung', 'entlasten', 'entlastung', 'entlastungen', 'entlastungspolitik',
                              'einkommen', 'heizkostenzuschuss', '9-euro-ticket', '9euroticket', '9€ticket',
                              'progression', 'armut', 'doppelwumms', 'doppel-wumms', 'energiegeld',
                              'einmalzahlung'],
    
    'sozialesKinder' : ['kinder', 'kindern', 'schule', 'schulen', 'eltern', 'kinderarmut',
                        'kindergrundsicherung', 'bildung', 'kindergeld', 'elterngeld',
                        'kinderbetreuung', 'jugendliche'],
    
    'sozialesAllgemein' : ['mindestlohn', 'übergewinnsteuer', 'schuldenbremse', 'armut', 'sorgen',
                           'aufstocken', 'aufstocker', 'aufstockerin', 'hartziv', 'hartz4', 'bürgergeld',
                           'ungleichheit', 'ungerechtigkeit', 'arbeitslosigkeit', 'erwerbstätigkeit',
                           'erwerbstätige', 'erwerbstätig', 'arbeitslos', 'arbeitslose', 'arbeitslosenquote',
                           'rente', 'renten', 'rentenkasse', 'rentenversicherung', 'sozialpolitischer',
                           'rentenbeiträge', 'rentensystem', 'aktienrente', 'bürgergeld-reform',
                           'bürgergeldreform', 'buergergeld', 'buergergeld-reform', 'buergergeldreform',
                           'sozialstaat', 'sozialpolitik', 'regelsätze', 'regelsaetze', 'sozialpolitisch',
                           'sozialpolitische', 'regelsatz', 'hartz-iv-sanktionen', 'hartz-4-sanktionen',
                           'grundeinkommen', 'zuverdienst', 'zuverdienstmöglichkeiten', 'hartz-iv',
                           'hartz-4', 'reichensteuer', 'vermögensteuer', 'vermögenssteuer', 'vermögensabgabe',
                           'superreiche', 'superreich', 'taxtherich', 'oberschicht', 'unterschicht',
                           'armutsgrenze'],
    
    'energieNordstream' : ['pipeline', 'nordstream', 'nordstream2', 'klimafonds', 'klimastiftung',
                           'klimastiftungmv', 'gazprom-stiftung', 'nordstream1', 'nordstreamii',
                           'nordstreami', 'ostseepipeline'],
    
    'energieAtomdiskurs' : ['atomkraft', 'atomkraftwerk', 'atomkraftwerke', 'kernkraft', 'kernenergie',
                            'kernkraftwerk', 'kernkraftwerke', 'akw', 'akws', 'netz', 'nuclear',
                            'laufzeitverlängerung', 'laufzeitverlaengerung', 'isar2', 'atomenergie'],
    
    'energieAllgemein' : ['energiepolitik', 'energie', 'gas', 'strom', 'öl',
                          'erneuerbare', 'erneuerbaren', 'energien', 'ctkwh', 'inflation', 'raffinerie',
                          'flüssiggas', 'lng', 'terminal', 'erdgas', 'erdöl',
                          'kohle', 'kohlekraftwerke', 'fracking', 'fossile', 'fossilen', 'pck', 'schwedt'],
    
    'energieKrise' : ['energiekrise', 'gaskrise', 'gasspeicher', 'gaspreis', 'gaspreise', 'versorgungssicherheit',
                      'energieknappheit', 'ölkrise', 'ölpreis', 'ölpreise', 'wärmepreisbremse',
                      'strompreisbremse', 'gaspreisbremse', 'gasumlage', 'energiekosten', 'gaspreisdeckel',
                      'energielieferungen', 'energiekosten', 'energiesicherheit', 'doppelwumms', 'doppel-wumms',
                      'energiekrieg'],
    
    'ukraineBewaffnung' : ['waffen', 'waffenlieferungen', 'panzer', 'kampfpanzer', 'schützenpanzer', 'gepard',
                           'marder', 'fuchs', 'leopard', 'munition', 'militärhilfe', 'panzerhaubitzen'],
    
    'ukraineReaktion' : ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine',
                         'mariupol', 'butscha', 'saporischschja', 'cherson', 'besatzer', 'aggressor',
                         'völkerrechtswidrig', 'völkerrechtswidriger', 'völkerrechtswidrigen',
                         'völkerrechtswidrigem', 'überfall'],
    
    'ukraineAllgemein' : ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia',
                          'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj',
                          'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato',
                          'söldner', 'söldnergruppe', 'prigoschin', 'prigozhin', 'lawrow', 'peskow'],
    
    'covidVirus' : ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'longcovid',
                    'gesundheitsminister', 'lauterbach', 'gesundheitssystem', 'epidemie', 'corona-krise'],
    
    'covidImpfung' : ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer',
                      'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe',
                      'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen',
                      'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin',
                      'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft',
                      'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen',
                      '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'],
    
    'covidInfektionsgeschehen' : ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage',
                                  'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen',
                                  'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit',
                                  'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz',
                                  '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen',
                                  'alpha', 'omikron', 'delta'],
    
    'covidMaßnahmen' : ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren', 'sperrstunde',
                        'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich',
                        'distancing', 'lockdown', 'shutdown', 'ffp2', 'ffp2-maske', 'ffp2-masken', 'quarantäne',
                        'isolation', 'selbstisolation', 'isolieren', 'schutzmaßnahmen', 'coronatest',
                        'coronatests', 'corona-test', 'corona-tests', 'lockerung', 'lockerungen',
                        'corona-maßnahme', 'corona-maßnahmen'],
    
    'verkehrAutomobil' : ['pkw', 'auto', 'autos', 'automobil', 'adblue', 'efuel', 'efuels', 'e-fuel', 'e-fuels',
                          'elektromobilität', 'emobilität', 'e-mobilität', 'emobility', 'e-mobility', 'e-auto',
                          'e-fahrzeug', 'e-autos', 'e-fahrzeuge', 'eauto', 'efahrzeug', 'eautos', 'efahrzeuge',
                          'elektroauto', 'elektroautos', 'plugin', 'ladeinfrastruktur', 'ladesäule', 'ladesäulen',
                          'autofahrende', 'autofahrer', 'autofahrerin', 'autofahrerinnen', 'antriebswende',
                          'automobilhersteller', 'verbrenner', 'verbrennungsmotor', 'verbrennerverbot',
                          'verkehrsverbund', 'verkehrsverbünde', 'iaa', 'autobahn', 'autobahnen', 'tempolimit',
                          'sonntagsfahrverbot', 'sprit', 'benzin', 'autofrei', 'autofreie', 'elektromobilitaet',
                          'emobilitaet'],
    
    'verkehrÖpnv' : ['öpnv', 'bus', 'busse', 'bahn', 'db', 'bahnhof', 'bahnhöfe', 'zug', 'züge', 'schiene',
                     'schienennetz', 'neuneuroticket', 'neuneurotickets', '9euroticket', '9-euro-ticket',
                     '9€ticket', '9eurotickets', '9-euro-tickets', '9€tickets', 'deutschlandticket',
                     'deutschlandtickets', 'anbindung', 'öpnv-nutzer', 'öpnv-nutzerin', 'öpnv-nutzende',
                     'bahnfahrende', 'zugfahrende', 'nahverkehr', 'deutschlandtakt', 'deutschland-takt',
                     'verspätung', 'verspätungen', 'zugausfall', 'zugausfälle', 'schienenersatzverkehr',
                     'deutschebahn', 'oepnv', 'oepnv-nutzer', 'oepnv-nutzerin', 'oepnv-nutzende'],
    
    'verkehrPolitik' : ['verkehr', 'verkehrswende', 'verkehrsminister', 'verkehrsministerium',
                        'verkehrsministers', 'verkehrsministeriums', 'mobilitätswende',
                        'bundesverkehrsminister', 'bundesverkehrsministerium', 'bundesverkehrsministers',
                        'bundesverkehrsministeriums', 'verkehrspolitisch', 'verkehrspolitischer',
                        'verkehrspolitische', 'verkehrspolitischen', 'verkehrspolitik', 'maut'],
    
    'verkehrFahrrad' : ['radfahrer', 'radfahrerin', 'radfahrerinnen', 'radfahren', 'fahrradfahrer',
                        'fahrradfahrerin', 'fahrradfahrerinnen', 'fahrrad', 'fahrradweg', 'fahrradwege',
                        'radweg', 'radwege', 'radfahrende', 'fahrradfreundlich', 'fahrradfreundliche',
                        'radfreundlich', 'radfreundliche', 'fahrradparkhäuser', 'fahrradstellplätze',
                        'radfahrens', 'fahrradfahrens', 'lastenrad', 'lastenfahrrad', 'lastenräder',
                        'lastenfahrräder'],
    
    'pluralismusMedien' : ['medien', 'rechtsstaat', 'pressefreiheit', 'presse', 'journalismus', 'journalist',
                           'journalistin', 'journalistinnen', 'journalisten', 'reporter', 'reporterin',
                           'reporterinnen', 'meinungsfreiheit', 'zensur', 'berichterstattung'],
    
    'zukunftVersprechen' : ['zukunftsvision', 'digitalstrategie', 'digitalisierung', 'innovation',
                            'entwicklung', 'bildung', 'transformation', 'digital', 'digitale', 'digitales',
                            'digitaler', 'idee', 'ideen', 'entbürokratisierung', 'energiewende',
                            'verkehrswende', 'klimaschutz', 'arbeitsplätze'],
    
    'zukunftEntwicklung' : ['zukunft', 'forschung', 'wissenschaft', 'herausforderung', 'herausforderungen',
                            'wohlstand', 'chance', 'chancen'],
    
    'verfassungsfeindlich' : ['verschwörungstheorien', 'verschwörungstheorie', 'verschwörungstheoretikern',
                              'verschwörungstheoretiker', 'verschwörungstheoretikerin',
                              'verschwörungstheoretikerinnen', 'telegram', 'reichsbürgern', 'reichsbürger',
                              'reichsbürgerin', 'reichsbürgerinnen', 'querdenkern', 'querdenker',
                              'querdenkerin', 'querdenkerinnen', 'querdenken', 'corona-protest',
                              'corona-proteste', 'corona-demonstration', 'corona-demonstrationen',
                              'coronaprotest', 'coronaproteste', 'coronademonstration', 'coronademonstrationen',
                              'corona-demo', 'corona-demos', 'coronademo', 'coronademos', 'impfskepsis',
                              'impfgegnern', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'coronaleugner',
                              'coronaleugnerin', 'coronaleugnerinnen', 'fake', 'fake-news', 'fakenews',
                              'querfront', 'hetze', 'extremismus', 'agitator', 'agitatoren', 'agitatorin',
                              'agitatorinnen', 'troll', 'trolle', 'trollfabrik', 'querdenken-szene',
                              'querdenkenszene', 'querdenkerszene', 'querdenker-szene', 'querdenkerecke',
                              'querdenker-ecke', 'querdenker-bewegung', 'querdenkerbewegung', 'verfassungsfeind',
                              'verfassungsfeinde', 'verfassungsfeindlich', 'montagsspaziergänger', 'impfpanik',
                              'ballweg', 'schwurbler', 'schwurblerin', 'schwurblerinnen',
                              'querdenker-spaziergang', 'rechtsextreminst', 'rechtsextreminsten',
                              'rechtextremistin', 'rechtsextremistinnen', 'verschwörungsszene',
                              'verschwörungs-szene', 'pegida', 'pegida-aktivisten', 'pegida-demo', 'qanon',
                              'qanon-szene', 'qanon-bewegung', 'qanonszene', 'qanonbewegung', 'desinformation',
                              'disinformation', 'verschwörungsgruppe', 'verschwörungsgruppen',
                              'verschwörungsgruppierung', 'verschwörungsgruppierungen', 'hildmann',
                              'naidoo', 'wendler']
}

*Die einzelnen Terms werden erstellt, d. h. es werden Begriffe aus der obigen Zusammenstellung extrahiert und dann mit einem Thema zusammen in Form eines Pandas-Datensatzes gespeichert.*

In [4]:
dfsThemenliste = []
# stemmer=Stemmer.Stemmer('de')

for key,values in themenliste.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsThemenliste.append(words)

termsThemenliste = pd.concat(dfsThemenliste)
termsThemenliste

,category,term
0,protesteIran,iran
1,protesteIran,iraner
2,protesteIran,iranerin
3,protesteIran,iranerinnen
4,protesteIran,iranisch
...,...,...
85,verfassungsfeindlich,verschwörungsgruppierung
86,verfassungsfeindlich,verschwörungsgruppierungen
87,verfassungsfeindlich,hildmann
88,verfassungsfeindlich,naidoo


*Initialisieren der Themenliste und Bilden des Vektorisierers, der die Themenliste aufgreifen und automatisiert auszählen und zuweisen wird.*

In [5]:
# Take the 'term' column from our list of terms
termlistThemenliste = list(set(termsThemenliste.term))

# binary=True oder False: bei True werden nur 1 oder 0 gemeldet wenn Wörter vorhanden sind
# vocabulary: die liste mit wörtern die erkannt werden sollen
vectorizer = TfidfVectorizer(binary=True, vocabulary = termlistThemenliste)

## BADEN-WÜRTTEMBERG (MEDIEN)

In [6]:
matrixBawüMedia = vectorizer.fit_transform(bawü.text.astype('U'), )
words_df_BawüMedia = pd.DataFrame(matrixBawüMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [7]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    bawü[category_name] = words_df_BawüMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: ukraineAllgemein 
Worte: ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia', 'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj', 'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato', 'söldner', 'söldnergruppe', 'prigoschin', 'prigozhin', 'lawrow', 'peskow'] 

Topic: ukraineBewaffnung 
Worte: ['waffen', 'waffenlieferungen', 'panzer', 'kampfpanzer', 'schützenpanzer', 'gepard', 'marder', 'fuchs', 'leopard', 'munition', 'militärhilfe', 'panzerhaubitzen'] 

Topic: ukraineReaktion 
Worte: ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine', 'mariupol', 'butscha', 'saporischschja', 'cherson', 'besatzer', 'aggressor', 'völkerrechtswidrig', 'völkerrechtswidriger', 'völkerrechtswidrigen', 'völkerrechtswidrigem', 'überfall'] 

Topic: verfassungsfeindlich 
Worte: ['verschwörungstheorien', 'verschwörungstheorie', 'verschwörungstheoretikern', 'verschwörungstheoretiker', 'ver

In [8]:
bawü.groupby('user').covidInfektionsgeschehen.sum().sort_values(ascending=False)

user
StZ_NEWS           288
StN_News           265
schwarzwaelder     262
RNZonline          196
remszeitung        131
badischezeitung    108
mittelbadische      78
BNN_BaNeuNa         55
online_MM           51
pznews              48
kanews              47
Schwaebische        34
Pressehaus          31
Suedkurier_News     22
stimmeonline        12
LKZ_online           7
Der_Teckbote         7
Tagblatt             5
ez_online            5
SchwaePo             1
MZ_nachrichten       1
mainecho_de          1
Gaeubote             1
GT_Gmuend            0
LKZ_Leonberg         0
KontextWZ            0
ZAK_Redaktion        0
SWPde                0
wnoz                 0
Name: covidInfektionsgeschehen, dtype: int64

In [9]:
bawü.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
BNN_BaNeuNa,2022-10-31T15:11:58Z2022-10-31T14:25:38Z2022-1...,6.623215e+21,Nach der vierten Niederlage in Reihe rückt die...,1679,1295,5005,372,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,9,20,28,107,32,20,88,7,50,61
Der_Teckbote,2022-10-31T15:07:30Z2022-10-31T15:07:30Z2022-1...,4.634149e+21,Neues aus #Polizeiberichte #Polizei #Teckbote ...,48,23,271,46,0,0.0,0.0,...,0,2,2,33,12,9,20,2,21,17
GT_Gmuend,2022-10-31T20:00:36Z2022-10-31T19:01:04Z2022-1...,4.308523e+20,"Bei der Gründung eine gute Idee, heute dringen...",5,2,14,2,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,0,8,2,1,2,0,4,5
Gaeubote,2022-10-31T12:05:13Z2022-10-30T11:51:49Z2022-1...,5.745758e+20,https://t.co/fAQzqoC9dFhttps://t.co/T9SE7kTNMU...,9,9,52,9,0,0.0,0.0,...,0,0,0,20,2,1,6,0,0,0
KontextWZ,2022-10-28T12:17:41Z2022-10-28T12:17:32Z2022-1...,9.759804e+19,Der Text: https://t.co/yf1N4jqCcoGewerkschafte...,59,94,362,18,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,2,0,0,0,0,1,3,2
LKZ_Leonberg,2022-10-07T17:24:05Z2022-10-07T14:23:12Z2022-1...,8.189215e+20,Eklat im Rathaus: Leonberger OB wehrt sich geg...,13,19,71,3,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,0,26,3,3,7,0,3,4
LKZ_online,2022-10-27T09:00:00Z2022-10-27T08:15:00Z2022-1...,1.773231e+21,Gutachter warnen: Die Klimakrise betrifft auch...,136,179,562,47,0,0.0,0.0,...,2,3,9,37,7,16,41,6,12,18
MZ_nachrichten,2022-08-01T05:04:32Z2022-07-31T14:36:35Z2022-0...,7.494984e+20,Marbachs Partnerschaft mit Tongling: Kunst sta...,6,17,153,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,0,8,6,7,11,0,4,12
Pressehaus,2022-10-31T22:00:34Z2022-10-31T21:01:08Z2022-1...,7.703295e+21,SG Heldenfingen/Heuchlingen mischt wieder mit ...,29,23,112,7,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,2,64,2,106,29,12,30,1,24,38


In [10]:
bawüCategorized = bawü
bawüCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/BAWÜ_medienKategorisiert.csv")

## BAYERN (MEDIEN)

In [11]:
matrixBayernMedia = vectorizer.fit_transform(bayern.text.astype('U'), )
words_df_BayernMedia = pd.DataFrame(matrixBayernMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [12]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    bayern[category_name] = words_df_BayernMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verkehrAutomobil 
Worte: ['pkw', 'auto', 'autos', 'automobil', 'adblue', 'efuel', 'efuels', 'e-fuel', 'e-fuels', 'elektromobilität', 'emobilität', 'e-mobilität', 'emobility', 'e-mobility', 'e-auto', 'e-fahrzeug', 'e-autos', 'e-fahrzeuge', 'eauto', 'efahrzeug', 'eautos', 'efahrzeuge', 'elektroauto', 'elektroautos', 'plugin', 'ladeinfrastruktur', 'ladesäule', 'ladesäulen', 'autofahrende', 'autofahrer', 'autofahrerin', 'autofahrerinnen', 'antriebswende', 'automobilhersteller', 'verbrenner', 'verbrennungsmotor', 'verbrennerverbot', 'verkehrsverbund', 'verkehrsverbünde', 'iaa', 'autobahn', 'autobahnen', 'tempolimit', 'sonntagsfahrverbot', 'sprit', 'benzin', 'autofrei', 'autofreie', 'elektromobilitaet', 'emobilitaet'] 

Topic: verkehrFahrrad 
Worte: ['radfahrer', 'radfahrerin', 'radfahrerinnen', 'radfahren', 'fahrradfahrer', 'fahrradfahrerin', 'fahrradfahrerinnen', 'fahrrad', 'fahrradweg', 'fahrradwege', 'radweg', 'radwege', 'radfahrende', 'fahrradfreundlich', 'fahrradfreundliche', 'r

In [13]:
bayern.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
merkur_de          74
tzmuenchen         59
pnp                27
AZ_Augsburg        26
mainpost           23
donaukurier        21
idowa              19
onetz_de           14
mz_de              11
AZ_Allgaeu          9
wochenblattnews     8
heimatzeitungen     6
NN_Online           6
sonntagsblatt       5
NZ_Online           4
bt_tagblatt         2
GZ_AKTUELL          1
fraenkischertag     1
hallo_muenchen      0
Staatszeitung       0
MZ_Online           0
HZOnline            0
Abendzeitung        0
Name: verfassungsfeindlich, dtype: int64

In [14]:
bayern.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
AZ_Allgaeu,2022-10-31T21:19:55Z2022-10-31T20:53:19Z2022-1...,2.042545e+22,Ina Müller in der Big Box Kempten: Lachen über...,94,241,727,138,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,11,9,9,473,92,36,121,17,117,106
AZ_Augsburg,2022-10-31T20:00:00Z2022-10-31T19:00:00Z2022-1...,1.186378e+22,Die Ursprünge von #RedBull gehen auf #Thailand...,4926,3765,14355,1003,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,81,122,26,104,9,20,117,88,149,117
Abendzeitung,2022-10-28T17:00:02Z2022-10-06T10:10:27Z2022-0...,3.345778e+19,AZ goes green: Unsere morgen erscheinende Woch...,27,27,94,12,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
GZ_AKTUELL,2022-10-19T06:44:04Z2022-10-15T15:00:46Z2022-1...,6.576437e+19,TV Bayern live hat von unserem 9. Bayerischen ...,5,35,78,12,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,1,0,0,0,0,0,0,2
HZOnline,2022-10-31T13:25:59Z2022-10-31T07:49:41Z2022-1...,1.314238e+21,Ehre für die ersten bayerischen Meister im Ink...,12,32,113,2,0,0.0,0.0,...,1,2,0,4,1,0,5,1,19,12
MZ_Online,2022-10-24T14:24:43Z2022-10-06T15:42:56Z2022-0...,4.424295e+19,Bürgerinnen und Bürger in #Türkheim müssen von...,4,3,14,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
NN_Online,2022-10-31T08:52:07Z2022-10-30T20:51:25Z2022-1...,8.046665e+20,Vor Gericht muss der #Drachenlord diesmal nich...,552,440,1677,96,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,3,3,6,12,0,5,14,1,5,10
NZ_Online,2022-10-22T08:12:58Z2022-10-09T15:34:40Z2022-0...,2.229495e+20,Der #FCN sorgt für ein Novum im deutschen Prof...,42,21,261,11,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,2,4,1,1,4,9,1,0,4
Staatszeitung,2022-10-27T14:33:41Z2022-10-21T09:23:53Z2022-0...,5.513332e+19,Ist der Landtags-Untersuchungsausschuss zur zw...,67,42,157,12,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,1,0,2,0,0,1,1,0,2


In [15]:
bayernCategorized = bayern
bayernCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/BAYERN_medienKategorisiert.csv")

## BRANDENBURG (MEDIEN)

In [16]:
matrixBrandenburgMedia = vectorizer.fit_transform(brandenburg.text.astype('U'), )
words_df_BrandenburgMedia = pd.DataFrame(matrixBrandenburgMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [17]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    brandenburg[category_name] = words_df_BrandenburgMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verteidigungspolitik 
Worte: ['zeitenwende', 'modernisieren', 'modernisierung', 'verteidigungspolitik', 'verteidigungsministerin', 'verteidigungsminister', 'verteidigungsministerium', 'sondervermögen', 'bundeswehr', 'truppe', 'heer', 'lambrecht', 'pistorius', 'verteidigungsfähig', 'verteidigungsfähigkeit', 'landesverteidigung'] 

Topic: zukunftEntwicklung 
Worte: ['zukunft', 'forschung', 'wissenschaft', 'herausforderung', 'herausforderungen', 'wohlstand', 'chance', 'chancen'] 

Topic: zukunftVersprechen 
Worte: ['zukunftsvision', 'digitalstrategie', 'digitalisierung', 'innovation', 'entwicklung', 'bildung', 'transformation', 'digital', 'digitale', 'digitales', 'digitaler', 'idee', 'ideen', 'entbürokratisierung', 'energiewende', 'verkehrswende', 'klimaschutz', 'arbeitsplätze'] 



In [18]:
brandenburg.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
maz_online    35
mozde         26
PNN_de         6
lr_online      6
NCasnik        0
Name: verfassungsfeindlich, dtype: int64

In [19]:
brandenburg.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
NCasnik,2022-10-26T10:53:11Z2022-10-21T09:59:57Z2022-1...,1.594598e+20,Nowy Casnik jo był ducy we #Wětošow_je we tamn...,31,18,126,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,0,0,0,0,2,0,0,1
PNN_de,2022-10-29T16:54:40Z2022-10-28T18:06:15Z2022-1...,3.415527e+21,Nach der Brei-Attacke auf ein #Monet-Gemälde ö...,972,1898,7495,531,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,5,14,6,37,19,10,50,6,26,32
lr_online,2022-10-31T18:30:00Z2022-10-31T17:30:00Z2022-1...,5.884661e+21,In den Abendstunden herrscht zwischen #Forstla...,418,963,1681,98,0,0.0,0.0,...,9,8,6,130,20,28,109,11,114,105
maz_online,2022-10-31T20:17:21Z2022-10-31T18:35:32Z2022-1...,7.717815e+21,"Die #LetzteGeneration beteuert, dass die „Sich...",1118,1167,4489,307,[Gif(thumbnailUrl='https://pbs.twimg.com/tweet...,0.0,0.0,...,19,41,35,166,33,43,139,17,71,95
mozde,2022-10-31T20:32:00Z2022-10-31T19:22:00Z2022-1...,7.521912e+21,In der Burgstraße in #Storkow finden wieder Ba...,314,528,1249,111,0,0.0,0.0,...,9,10,26,266,33,104,185,8,90,75


In [20]:
brandenburgCategorized = brandenburg
brandenburgCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/BRANDENBURG_medienKategorisiert.csv")

## BERLIN (MEDIEN)

In [21]:
matrixBerlinMedia = vectorizer.fit_transform(berlin.text.astype('U'), )
words_df_BerlinMedia = pd.DataFrame(matrixBerlinMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [22]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    berlin[category_name] = words_df_BerlinMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: ukraineAllgemein 
Worte: ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia', 'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj', 'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato', 'söldner', 'söldnergruppe', 'prigoschin', 'prigozhin', 'lawrow', 'peskow'] 

Topic: ukraineBewaffnung 
Worte: ['waffen', 'waffenlieferungen', 'panzer', 'kampfpanzer', 'schützenpanzer', 'gepard', 'marder', 'fuchs', 'leopard', 'munition', 'militärhilfe', 'panzerhaubitzen'] 

Topic: ukraineReaktion 
Worte: ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine', 'mariupol', 'butscha', 'saporischschja', 'cherson', 'besatzer', 'aggressor', 'völkerrechtswidrig', 'völkerrechtswidriger', 'völkerrechtswidrigen', 'völkerrechtswidrigem', 'überfall'] 

Topic: verfassungsfeindlich 
Worte: ['verschwörungstheorien', 'verschwörungstheorie', 'verschwörungstheoretikern', 'verschwörungstheoretiker', 'ver

In [23]:
berlin.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
Tagesspiegel       125
berlinerzeitung    111
BERLINER_KURIER     66
ndaktuell           58
Junge_Freiheit      42
bzberlin            37
JuedischeOnline     36
morgenpost          30
Jungle_World        17
vorwaerts           17
TspCheckpoint       10
berlinerwoche        2
BerlAbendblatt       0
pukzeitung           0
tag_des_herrn        0
Name: verfassungsfeindlich, dtype: int64

In [24]:
berlin.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
BERLINER_KURIER,2022-10-31T22:38:57Z2022-10-31T22:17:27Z2022-1...,1.752262e+22,Unsere Kolumnistin hat diesen dahingeschrieben...,4017,5696,11306,698,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,45,129,66,212,32,19,143,39,111,68
BerlAbendblatt,2022-10-31T15:52:37Z2022-10-31T14:38:04Z2022-1...,6.904215e+20,Der Oktober war warm... zu warm? Sonne und mil...,50,128,268,29,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,1,0,0,11,6,8,26,0,6,6
JuedischeOnline,2022-10-31T16:59:33Z2022-10-31T15:46:26Z2022-1...,5.260342e+21,Nach einem Gespräch mit dem Kölner Polizeipräs...,7652,30880,126633,3231,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,15,41,36,4,0,0,7,15,50,31
Junge_Freiheit,2022-10-31T17:49:52Z2022-10-31T17:05:56Z2022-1...,4.318172e+21,Der Verfassungsschutz betreibt Hunderte rechts...,39100,89350,301591,7992,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,0.0,...,39,60,42,60,2,11,21,60,39,56
Jungle_World,2022-10-31T08:08:21Z2022-10-30T14:30:39Z2022-1...,7.101595e+20,Der Pragmatismus ihres Führungspersonals könnt...,297,2612,7513,324,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,3,22,17,2,0,0,1,3,6,9
Tagesspiegel,2022-10-31T22:20:37Z2022-10-31T21:26:41Z2022-1...,2.856520e+22,Für seinen Rassismus gegen Araber wurde er ger...,116599,106743,333712,30342,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,317,598,125,383,111,122,437,221,213,274
TspCheckpoint,2022-10-31T10:04:50Z2022-10-31T09:39:29Z2022-1...,1.641941e+21,Ausgabe von #Urkunden momentan nicht möglich: ...,2766,5907,12524,948,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,9,4,10,34,16,26,46,6,24,35
berlinerwoche,2022-10-31T23:01:30Z2022-10-31T22:42:23Z2022-1...,1.325534e+22,Übergriffe in Schwimmbädern: Grüne und SPD wol...,269,1040,2046,191,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,1,0,2,58,86,16,103,7,84,113
berlinerzeitung,2022-10-31T22:55:53Z2022-10-31T22:41:14Z2022-1...,2.649360e+22,In dem Werk des Apple-Zulieferers Foxconn arbe...,200641,161206,490220,33723,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,228,372,111,376,74,97,448,156,174,206


In [25]:
berlinCategorized = berlin
berlinCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/BERLIN_medienKategorisiert.csv")

## BREMEN

In [26]:
matrixBremenMedia = vectorizer.fit_transform(bremen.text.astype('U'), )
words_df_BremenMedia = pd.DataFrame(matrixBremenMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [27]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    bremen[category_name] = words_df_BremenMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

In [28]:
bremen.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
weserkurier       40
nordseezeitung     1
Wuemme_Zeitung     0
Name: verfassungsfeindlich, dtype: int64

In [29]:
bremen.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
Wuemme_Zeitung,2022-10-19T16:37:15Z2022-09-18T17:56:33Z2022-0...,1.068108e+19,An der Klinik #Lilienthal veranstalten die @FF...,10,2,3,1,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
nordseezeitung,2022-10-14T17:17:31Z2022-10-13T16:08:51Z2022-1...,1.180491e+20,"Mehr als 100 Jobs im Hafen von #Bremerhaven, L...",15,58,93,5,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,1,1,1,3,0,1,1,0,0,5
weserkurier,2022-10-31T22:46:07Z2022-10-31T22:19:48Z2022-1...,1.336942e+22,Andreas Seebeck erforscht die Kraft der Worte ...,3823,3111,14161,933,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,45,88,40,148,49,57,192,39,169,180


In [30]:
bremenCategorized = bremen
bremenCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/BREMEN_medienKategorisiert.csv")

## HAMBURG

In [31]:
matrixHamburgMedia = vectorizer.fit_transform(hamburg.text.astype('U'), )
words_df_HamburgMedia = pd.DataFrame(matrixHamburgMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [32]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    hamburg[category_name] = words_df_HamburgMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verkehrÖpnv 
Worte: ['öpnv', 'bus', 'busse', 'bahn', 'db', 'bahnhof', 'bahnhöfe', 'zug', 'züge', 'schiene', 'schienennetz', 'neuneuroticket', 'neuneurotickets', '9euroticket', '9-euro-ticket', '9€ticket', '9eurotickets', '9-euro-tickets', '9€tickets', 'deutschlandticket', 'deutschlandtickets', 'anbindung', 'öpnv-nutzer', 'öpnv-nutzerin', 'öpnv-nutzende', 'bahnfahrende', 'zugfahrende', 'nahverkehr', 'deutschlandtakt', 'deutschland-takt', 'verspätung', 'verspätungen', 'zugausfall', 'zugausfälle', 'schienenersatzverkehr', 'deutschebahn', 'oepnv', 'oepnv-nutzer', 'oepnv-nutzerin', 'oepnv-nutzende'] 

Topic: verteidigungspolitik 
Worte: ['zeitenwende', 'modernisieren', 'modernisierung', 'verteidigungspolitik', 'verteidigungsministerin', 'verteidigungsminister', 'verteidigungsministerium', 'sondervermögen', 'bundeswehr', 'truppe', 'heer', 'lambrecht', 'pistorius', 'verteidigungsfähig', 'verteidigungsfähigkeit', 'landesverteidigung'] 

Topic: zukunftEntwicklung 
Worte: ['zukunft', 'for

In [33]:
hamburg.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
abendblatt    96
mopo          47
Name: verfassungsfeindlich, dtype: int64

In [34]:
hamburg.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
abendblatt,2022-10-31T21:17:47Z2022-10-31T21:07:17Z2022-1...,6.948173e+22,Proteste gegen Energiepolitik in MV: Weniger L...,6228,4746,23840,1463,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,65,111,96,1193,212,382,685,92,267,343
mopo,2022-10-31T22:42:26Z2022-10-31T21:38:51Z2022-1...,1.327965e+22,Hamburger Winternotprogramm beginnt – doch es ...,14859,8072,50344,2437,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,22,36,47,504,83,30,203,10,24,33


In [35]:
hamburgCategorized = hamburg
hamburg.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/HAMBURG_medienKategorisiert.csv")

## HESSEN

In [36]:
matrixHessenMedia = vectorizer.fit_transform(hessen.text.astype('U'), )
words_df_HessenMedia = pd.DataFrame(matrixHessenMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [37]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    hessen[category_name] = words_df_HessenMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verkehrAutomobil 
Worte: ['pkw', 'auto', 'autos', 'automobil', 'adblue', 'efuel', 'efuels', 'e-fuel', 'e-fuels', 'elektromobilität', 'emobilität', 'e-mobilität', 'emobility', 'e-mobility', 'e-auto', 'e-fahrzeug', 'e-autos', 'e-fahrzeuge', 'eauto', 'efahrzeug', 'eautos', 'efahrzeuge', 'elektroauto', 'elektroautos', 'plugin', 'ladeinfrastruktur', 'ladesäule', 'ladesäulen', 'autofahrende', 'autofahrer', 'autofahrerin', 'autofahrerinnen', 'antriebswende', 'automobilhersteller', 'verbrenner', 'verbrennungsmotor', 'verbrennerverbot', 'verkehrsverbund', 'verkehrsverbünde', 'iaa', 'autobahn', 'autobahnen', 'tempolimit', 'sonntagsfahrverbot', 'sprit', 'benzin', 'autofrei', 'autofreie', 'elektromobilitaet', 'emobilitaet'] 

Topic: verkehrFahrrad 
Worte: ['radfahrer', 'radfahrerin', 'radfahrerinnen', 'radfahren', 'fahrradfahrer', 'fahrradfahrerin', 'fahrradfahrerinnen', 'fahrrad', 'fahrradweg', 'fahrradwege', 'radweg', 'radwege', 'radfahrende', 'fahrradfreundlich', 'fahrradfreundliche', 'r

In [38]:
hessen.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
Echo_Online       38
mittelhessende    38
fnp_zeitung       33
wknachrichten     18
mspnachrichten    12
HNA_online        10
oberhessische      8
fuldaerzeitung     2
opmarburg          1
KAgezwitscher      0
boersenzeitung     0
Name: verfassungsfeindlich, dtype: int64

In [39]:
hessen.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
Echo_Online,2022-10-31T20:39:57Z2022-10-31T17:41:18Z2022-1...,1.855069e+22,Großeinsatz der Polizei im mittelhessischen Aß...,972,5202,4236,414,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,36,102,38,422,94,76,300,30,240,180
HNA_online,2022-10-31T17:45:00Z2022-10-31T15:01:01Z2022-1...,2.145395e+21,In einem abgelegenen Landgasthof im Sauerland ...,253,332,764,87,0,0.0,0.0,...,5,12,10,60,3,5,20,5,19,10
KAgezwitscher,2022-10-31T14:05:16Z2022-10-31T14:05:15Z2022-1...,1.314429e+21,Mit enormer Spielfreude führten rund 20 Kinder...,14,33,276,7,0,0.0,0.0,...,2,0,0,10,3,2,11,2,24,28
boersenzeitung,2022-10-31T17:02:50Z2022-10-31T16:08:06Z2022-1...,4.287275e+21,Die Stimmung in #Banken &amp; #Sparkassen ist ...,474,1439,2861,162,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,2,58,0,28,0,2,9,8,70,109
fnp_zeitung,2022-10-31T23:01:29Z2022-10-31T22:50:38Z2022-1...,2.607050e+22,Im Norden von Frankfurt soll der Windpark von ...,1166,1131,6832,370,0,0.0,0.0,...,11,14,33,334,90,63,269,21,229,238
fuldaerzeitung,2022-10-31T09:42:20Z2022-10-31T06:11:41Z2022-1...,1.706978e+21,#Fulda freut sich auf die #Landesgartenschau (...,364,147,423,51,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,0.0,...,4,2,2,58,8,7,18,3,17,9
mittelhessende,2022-10-31T18:57:07Z2022-10-31T17:21:52Z2022-1...,7.274984e+21,In Aßlar-Werdorf ist gerade ein Großaufgebot d...,327,3804,1295,129,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,18,40,38,121,27,40,106,18,88,77
mspnachrichten,2022-10-31T20:44:16Z2022-10-31T17:41:18Z2022-1...,3.348488e+21,Großeinsatz der Polizei im mittelhessischen Aß...,29,92,75,9,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,18,31,12,73,9,13,55,14,46,31
oberhessische,2022-10-31T18:57:08Z2022-10-31T17:41:20Z2022-1...,1.770809e+21,In Aßlar-Werdorf ist gerade ein Großaufgebot d...,30,193,71,6,0,0.0,0.0,...,10,26,8,33,4,5,30,10,16,14


In [40]:
hessenCategorized = hessen
hessenCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/HESSEN_medienKategorisiert.csv")

## MECKLENBURG-VORPOMMERN

In [41]:
matrixMeckpommMedia = vectorizer.fit_transform(mecklenburg.text.astype('U'), )
words_df_MeckpommMedia = pd.DataFrame(matrixMeckpommMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [42]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    mecklenburg[category_name] = words_df_MeckpommMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

In [43]:
mecklenburg.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
OZlive        21
Nordkurier     9
Name: verfassungsfeindlich, dtype: int64

In [44]:
mecklenburg.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
Nordkurier,2022-10-31T19:26:20Z2022-10-31T08:47:14Z2022-1...,2.001594e+21,Auch am Reformationstag gibt es wieder Montags...,1713,1552,3742,356,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,12,9,9,33,1,6,41,12,9,32
OZlive,2022-10-31T20:17:18Z2022-10-31T18:24:36Z2022-1...,7.984101e+21,Im Heimspiel gegen den SC Staaken unterliegt d...,1909,1953,9743,518,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,15,28,21,194,24,19,125,20,97,82


In [45]:
mecklenburgCategorized = mecklenburg
mecklenburgCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/MECKLENBURG_medienKategorisiert.csv")

## NIEDERSACHSEN

In [46]:
matrixNdsMedia = vectorizer.fit_transform(niedersachsen.text.astype('U'), )
words_df_NdsMedia = pd.DataFrame(matrixNdsMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [47]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    niedersachsen[category_name] = words_df_NdsMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verkehrAutomobil 
Worte: ['pkw', 'auto', 'autos', 'automobil', 'adblue', 'efuel', 'efuels', 'e-fuel', 'e-fuels', 'elektromobilität', 'emobilität', 'e-mobilität', 'emobility', 'e-mobility', 'e-auto', 'e-fahrzeug', 'e-autos', 'e-fahrzeuge', 'eauto', 'efahrzeug', 'eautos', 'efahrzeuge', 'elektroauto', 'elektroautos', 'plugin', 'ladeinfrastruktur', 'ladesäule', 'ladesäulen', 'autofahrende', 'autofahrer', 'autofahrerin', 'autofahrerinnen', 'antriebswende', 'automobilhersteller', 'verbrenner', 'verbrennungsmotor', 'verbrennerverbot', 'verkehrsverbund', 'verkehrsverbünde', 'iaa', 'autobahn', 'autobahnen', 'tempolimit', 'sonntagsfahrverbot', 'sprit', 'benzin', 'autofrei', 'autofreie', 'elektromobilitaet', 'emobilitaet'] 

Topic: verkehrFahrrad 
Worte: ['radfahrer', 'radfahrerin', 'radfahrerinnen', 'radfahren', 'fahrradfahrer', 'fahrradfahrerin', 'fahrradfahrerinnen', 'fahrrad', 'fahrradweg', 'fahrradwege', 'radweg', 'radwege', 'radfahrende', 'fahrradfreundlich', 'fahrradfreundliche', 'r

In [48]:
niedersachsen.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
SZLZ1              79
HAZ                43
neuepresse         37
goetageblatt       29
noz_de             24
BS_Zeitung         23
oz_online_de       21
ga_online          21
nwzonline          19
azgifhorn          14
wazwolfsburg       13
Rundblick_NDS      13
TAGEBLATTonline    11
sn_online          11
pazpeine           11
Kreisblatt         11
Dewezet            10
HarzKurier          7
on_online_de        6
lzgezwitscher       5
cezett              4
Kreiszeitung        4
Die_Harke           3
goslarsche          1
WZonline            0
kzw_redaktion       0
Harlinger1862       0
Wochenblatt1791     0
hinews              0
einmorgenpost       0
Name: verfassungsfeindlich, dtype: int64

In [49]:
niedersachsen.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
BS_Zeitung,2022-10-31T22:46:06Z2022-10-31T22:15:24Z2022-1...,9.955791e+21,So begehen Braunschweiger Kirchen das Fest der...,389,307,1722,145,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,6,10,23,212,50,33,112,9,67,95
Dewezet,2022-10-31T21:46:42Z2022-10-31T21:03:56Z2022-1...,1.517659e+22,Bei Galeria Karstadt Kaufhof droht über 40 Fil...,84,108,580,36,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,55,83,10,104,43,6,115,29,65,51
Die_Harke,2022-10-31T17:56:59Z2022-10-31T16:52:46Z2022-1...,5.264101e+21,Darüber spricht Nienburg am Abend https://t.co...,49,64,404,9,0,0.0,0.0,...,1,5,3,81,16,3,21,4,20,10
HAZ,2022-10-31T18:32:12Z2022-10-31T17:49:27Z2022-1...,9.294628e+21,Die Polizei versucht mit einer Öffentlichkeits...,10605,4599,32226,1958,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,24,34,43,269,75,59,184,48,75,91
Harlinger1862,2022-10-26T09:50:14Z2022-10-17T08:29:49Z2022-1...,1.507403e+20,Wie geht es beim insolventen Golfclub Ostfries...,3,16,45,1,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,1,0,1,0,0,0,0,1,0
HarzKurier,2022-10-27T09:57:01Z2022-10-27T06:11:53Z2022-1...,2.601286e+21,Restaurant Da Capo in Osterode soll mit neuem ...,46,317,450,14,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,1,2,7,70,5,11,22,1,26,23
Kreisblatt,2022-10-31T15:02:03Z2022-10-31T13:17:35Z2022-1...,6.723040e+21,Nach zweijähriger Pause: Nikolausfahrten mit „...,40,19,217,48,0,0.0,0.0,...,2,6,11,95,28,9,40,2,37,35
Kreiszeitung,2022-10-28T14:01:46Z2022-10-28T12:27:46Z2022-1...,1.623584e+21,„Raving Rishi“: Party-Video aus Ibiza sorgt fü...,250,141,460,50,0,0.0,0.0,...,6,11,4,21,2,1,14,4,3,2
Rundblick_NDS,2022-10-31T15:34:50Z2022-10-31T10:05:09Z2022-1...,1.555904e+21,EIL: Entgegen der ursprünglichen Planung werde...,266,587,1676,205,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,5,23,13,8,0,10,16,14,22,44


In [50]:
ndsCategorized = niedersachsen
ndsCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/NIEDERSACHSEN_medienKategorisiert.csv")

## NORDRHEIN-WESTFALEN

In [51]:
matrixNrwMedia = vectorizer.fit_transform(nrw.text.astype('U'), )
words_df_NrwMedia = pd.DataFrame(matrixNrwMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [52]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    nrw[category_name] = words_df_NrwMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: sozialesKinder 
Worte: ['kinder', 'kindern', 'schule', 'schulen', 'eltern', 'kinderarmut', 'kindergrundsicherung', 'bildung', 'kindergeld', 'elterngeld', 'kinderbetreuung', 'jugendliche'] 

Topic: sozialesWohnen 
Worte: ['wohnungsbau', 'wohnen', 'bauen', 'mieter', 'bauen', 'mieterinnen', 'mieterin', 'miete', 'mietendeckel', 'mietenstopp', 'sozialwohnung', 'sozialwohnungen'] 

Topic: ukraineAllgemein 
Worte: ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia', 'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj', 'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato', 'söldner', 'söldnergruppe', 'prigoschin', 'prigozhin', 'lawrow', 'peskow'] 

Topic: ukraineBewaffnung 
Worte: ['waffen', 'waffenlieferungen', 'panzer', 'kampfpanzer', 'schützenpanzer', 'gepard', 'marder', 'fuchs', 'leopard', 'munition', 'militärhilfe', 'panzerhaubitzen'] 

Topic: ukraineReaktion 
Worte: ['sanktionen', 'angriffskrieg'

In [53]:
nrw.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
wznewsline         93
rponline           43
WN_Redaktion       43
WAZ_Redaktion      41
gabonn             32
RN_DORTMUND        29
handelsblatt       21
westfalenblatt     19
KSTA               17
nwnews             16
MT_Online           5
DieGlocke           5
express24           4
DZ_Duelmen          4
soesteranzeiger     3
WA_online           2
DasSauerland        2
KoelnischeR         2
aachenerzeitung     1
lzonline            1
rgaonline           1
blickaktuell        0
UnsereZeit_UZ       0
UnsereKirche        0
SiegenerZeitung     0
handwerksblatt      0
Sauerlandkurier     0
SGTageblatt         0
KreisblattHalle     0
WRundschau          0
Name: verfassungsfeindlich, dtype: int64

In [54]:
nrw.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
DZ_Duelmen,2022-10-31T18:23:04Z2022-10-31T17:27:36Z2022-1...,2.068265e+21,Sirene im richtigen Moment: Obwohl GW Hausdülm...,44,45,165,22,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,2,0,4,53,19,4,42,0,21,22
DasSauerland,2022-10-30T13:54:36Z2022-10-30T13:54:34Z2022-1...,7.454687e+20,Polizeieinsatz: Dortmund: Schussabgaben als „E...,33,44,157,8,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,2,2,2,33,2,9,16,0,12,15
DieGlocke,2022-10-31T18:06:39Z2022-10-31T18:06:37Z2022-1...,4.748729e+21,Oberbürgermeister: „Gutes Pflaster für Diploma...,86,70,821,48,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,4,5,5,90,17,4,40,6,17,20
KSTA,2022-10-31T19:44:01Z2022-10-31T19:00:43Z2022-1...,8.257418e+21,"#Putin will Europas Gesellschaften zersetzen, ...",7090,4809,20192,1787,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,18,57,17,116,31,43,119,17,59,59
KoelnischeR,2022-10-31T09:44:34Z2022-10-31T06:32:20Z2022-1...,5.591053e+20,Anonymer Anrufer droht mit Straftat - Schulzen...,86,62,156,17,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,0.0,...,1,7,2,14,4,6,14,0,3,7
KreisblattHalle,2022-10-11T09:40:10Z2022-10-11T08:56:12Z2022-1...,1.559553e+21,New #OpenMicStory Unsichtbaren Geistern auf de...,7,10,139,2,0,0.0,0.0,...,1,0,0,34,13,5,6,0,9,7
MT_Online,2022-10-31T19:29:35Z2022-10-31T18:04:47Z2022-1...,3.914228e+21,Wie sich zwei Frauen nicht entmutigen lassen [...,254,213,435,106,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,1,3,5,34,4,3,21,4,21,14
RN_DORTMUND,2022-10-31T22:28:55Z2022-10-31T21:43:26Z2022-1...,8.303019e+21,RN+ Stadt rechnet mit Ansturm bei Wohngeld-Ant...,2977,7185,7000,1176,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,11,8,29,127,28,15,99,3,21,28
SGTageblatt,2022-10-23T14:37:31Z2022-10-18T11:00:02Z2022-0...,1.940234e+20,"@wagenbrn \nKorrekt, aber darum ging´s doch ga...",13,12,55,2,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,1,0,1,1,3,5,0,2,6


In [55]:
nrwCategorized = nrw
nrwCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/NRW_medienKategorisiert.csv")

## RHEINLAND-PFALZ

In [56]:
matrixRlpMedia = vectorizer.fit_transform(rlp.text.astype('U'), )
words_df_RlpMedia = pd.DataFrame(matrixRlpMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [57]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    rlp[category_name] = words_df_RlpMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verfassungsfeindlich 
Worte: ['verschwörungstheorien', 'verschwörungstheorie', 'verschwörungstheoretikern', 'verschwörungstheoretiker', 'verschwörungstheoretikerin', 'verschwörungstheoretikerinnen', 'telegram', 'reichsbürgern', 'reichsbürger', 'reichsbürgerin', 'reichsbürgerinnen', 'querdenkern', 'querdenker', 'querdenkerin', 'querdenkerinnen', 'querdenken', 'corona-protest', 'corona-proteste', 'corona-demonstration', 'corona-demonstrationen', 'coronaprotest', 'coronaproteste', 'coronademonstration', 'coronademonstrationen', 'corona-demo', 'corona-demos', 'coronademo', 'coronademos', 'impfskepsis', 'impfgegnern', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'coronaleugner', 'coronaleugnerin', 'coronaleugnerinnen', 'fake', 'fake-news', 'fakenews', 'querfront', 'hetze', 'extremismus', 'agitator', 'agitatoren', 'agitatorin', 'agitatorinnen', 'troll', 'trolle', 'trollfabrik', 'querdenken-szene', 'querdenkenszene', 'querdenkerszene', 'querdenker-szene', 'querdenkerecke', 'querden

In [58]:
rlp.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
aznachrichten    27
wznachrichten    17
Volksfreund      13
rheinpfalz        7
RheinZeitung      2
Name: verfassungsfeindlich, dtype: int64

In [59]:
rlp.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
RheinZeitung,2022-10-31T18:57:25Z2022-10-31T15:57:10Z2022-1...,4.172560e+21,Ein 30-Tonnen-Schiff auf dem Weg nach Belgien ...,862,942,2156,101,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,5,9,2,76,8,21,48,17,46,44
Volksfreund,2022-10-31T21:00:03Z2022-10-31T20:00:03Z2022-1...,5.378282e+21,Der letzte große deutsche Warenhauskonzern käm...,341,338,1340,92,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,8,13,13,173,22,13,62,10,53,35
aznachrichten,2022-10-31T21:29:13Z2022-10-31T20:38:42Z2022-1...,1.047963e+22,Unser Text zur Mitgliederversammlung von Mainz...,867,1232,3973,443,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,0.0,...,25,54,27,215,34,51,147,14,124,91
rheinpfalz,2022-10-15T16:37:47Z2022-10-15T13:27:42Z2022-1...,1.207096e+21,CDU-Chef Baldauf im Interview mit der RHEINPFA...,260,463,1753,141,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,3,1,7,30,2,2,15,2,3,3
wznachrichten,2022-10-31T16:41:02Z2022-10-31T16:19:07Z2022-1...,5.161675e+21,Die Besuchermassen im Wormser Tiergarten haben...,117,141,503,27,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,17,40,17,95,15,21,82,14,64,46


In [60]:
rlpCategorized = rlp
rlpCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/RHEINLAND_medienKategorisiert.csv")

## SAARLAND

In [61]:
matrixSaarlandMedia = vectorizer.fit_transform(saarland.text.astype('U'), )
words_df_SaarlandMedia = pd.DataFrame(matrixSaarlandMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [62]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    saarland[category_name] = words_df_SaarlandMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

In [63]:
saarland.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
szaktuell    24
Name: verfassungsfeindlich, dtype: int64

In [64]:
saarland.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
szaktuell,2022-10-31T21:33:02Z2022-10-31T21:00:03Z2022-1...,1.173403e+22,Unter dem Titel „Bella Figura“ vereint der neu...,2599,4486,11805,741,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,49,72,24,280,27,39,122,39,136,123


In [65]:
saarlandCategorized = saarland
saarlandCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/SAARLAND_medienKategorisiert.csv")

## SACHSEN

In [66]:
matrixSachsenMedia = vectorizer.fit_transform(sachsen.text.astype('U'), )
words_df_SachsenMedia = pd.DataFrame(matrixSachsenMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [67]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    sachsen[category_name] = words_df_SachsenMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: ukraineBewaffnung 
Worte: ['waffen', 'waffenlieferungen', 'panzer', 'kampfpanzer', 'schützenpanzer', 'gepard', 'marder', 'fuchs', 'leopard', 'munition', 'militärhilfe', 'panzerhaubitzen'] 

Topic: ukraineReaktion 
Worte: ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine', 'mariupol', 'butscha', 'saporischschja', 'cherson', 'besatzer', 'aggressor', 'völkerrechtswidrig', 'völkerrechtswidriger', 'völkerrechtswidrigen', 'völkerrechtswidrigem', 'überfall'] 

Topic: verfassungsfeindlich 
Worte: ['verschwörungstheorien', 'verschwörungstheorie', 'verschwörungstheoretikern', 'verschwörungstheoretiker', 'verschwörungstheoretikerin', 'verschwörungstheoretikerinnen', 'telegram', 'reichsbürgern', 'reichsbürger', 'reichsbürgerin', 'reichsbürgerinnen', 'querdenkern', 'querdenker', 'querdenkerin', 'querdenkerinnen', 'querdenken', 'corona-protest', 'corona-proteste', 'corona-demonstration', 'corona-demonstrationen', 'coronaprotest', 'coronaproteste', 'coronademons

In [68]:
sachsen.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
LIZ_de           128
saechsischeDE     60
LVZ               41
dnn_online        32
freie_presse      20
sonntagticker      2
Name: verfassungsfeindlich, dtype: int64

In [69]:
sachsen.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
LIZ_de,2022-10-31T23:49:48Z2022-10-31T22:41:43Z2022-1...,3.575950e+21,"@KleinerTrompet1 Nach jeder ""Freiheits""-Demo z...",1426,3751,14118,466,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,0.0,...,5,15,128,83,27,147,94,4,33,114
LVZ,2022-10-31T21:18:53Z2022-10-31T20:37:03Z2022-1...,9.062154e+21,Am #Reformationstag wurde in vielen mitteldeut...,4827,3004,19112,1406,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,25,37,41,169,45,56,131,20,106,95
dnn_online,2022-10-31T21:09:20Z2022-10-31T19:45:21Z2022-1...,8.673072e+21,Internetbetrug: 79-jähriger Dresdner verliert ...,1130,557,2513,460,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,14,18,32,157,77,12,94,5,22,37
freie_presse,2022-10-31T20:55:29Z2022-10-31T19:52:49Z2022-1...,4.508709e+21,Der letzte große deutsche Warenhauskonzern käm...,1774,2127,7564,972,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,15,24,20,70,6,14,64,15,36,35
saechsischeDE,2022-10-31T07:40:59Z2022-10-30T17:07:14Z2022-1...,8.566335e+21,#Dresden: 28-jähriger #Motorradfahrer stirbt n...,2821,4270,11080,1450,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,36,41,60,165,50,32,144,14,75,68
sonntagticker,2022-10-29T17:44:53Z2022-10-26T12:56:45Z2022-1...,6.297334e+20,Sachsens evangelischer Landesbischof Tobias Bi...,85,181,963,102,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,6,2,2,0,0,0,0,3,5,7


In [70]:
sachsenCategorized = sachsen
sachsenCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/SACHSEN_medienKategorisiert.csv")

## SACHSEN-ANHALT

In [71]:
matrixSachsenAnhaltMedia = vectorizer.fit_transform(sachsenAnhalt.text.astype('U'), )
words_df_SachsenAnhaltMedia = pd.DataFrame(matrixSachsenAnhaltMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [72]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    sachsenAnhalt[category_name] = words_df_SachsenAnhaltMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verkehrAutomobil 
Worte: ['pkw', 'auto', 'autos', 'automobil', 'adblue', 'efuel', 'efuels', 'e-fuel', 'e-fuels', 'elektromobilität', 'emobilität', 'e-mobilität', 'emobility', 'e-mobility', 'e-auto', 'e-fahrzeug', 'e-autos', 'e-fahrzeuge', 'eauto', 'efahrzeug', 'eautos', 'efahrzeuge', 'elektroauto', 'elektroautos', 'plugin', 'ladeinfrastruktur', 'ladesäule', 'ladesäulen', 'autofahrende', 'autofahrer', 'autofahrerin', 'autofahrerinnen', 'antriebswende', 'automobilhersteller', 'verbrenner', 'verbrennungsmotor', 'verbrennerverbot', 'verkehrsverbund', 'verkehrsverbünde', 'iaa', 'autobahn', 'autobahnen', 'tempolimit', 'sonntagsfahrverbot', 'sprit', 'benzin', 'autofrei', 'autofreie', 'elektromobilitaet', 'emobilitaet'] 

Topic: verkehrFahrrad 
Worte: ['radfahrer', 'radfahrerin', 'radfahrerinnen', 'radfahren', 'fahrradfahrer', 'fahrradfahrerin', 'fahrradfahrerinnen', 'fahrrad', 'fahrradweg', 'fahrradwege', 'radweg', 'radwege', 'radfahrende', 'fahrradfreundlich', 'fahrradfreundliche', 'r

In [73]:
sachsenAnhalt.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
mzwebde            42
Volksstimme        33
volksstimme_md      1
glaubeundheimat     0
Name: verfassungsfeindlich, dtype: int64

In [74]:
sachsenAnhalt.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
Volksstimme,2022-10-31T17:19:12Z2022-10-31T17:06:22Z2022-1...,1.707116e+22,Magdeburger Junioren holen sich WM-Bronze http...,2596,1328,10963,640,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,26,53,33,168,57,59,123,30,84,73
glaubeundheimat,2022-10-07T06:23:03Z2022-09-16T11:53:07Z2022-0...,6.655904e+19,Uns interessiert eure Meinung. Jetzt an der Le...,11,36,75,3,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
mzwebde,2022-10-31T19:39:52Z2022-10-31T18:46:08Z2022-1...,2.832682e+22,Sechs Verletzte bei Unfall auf B6 zwischen Göb...,2132,2256,9141,824,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,82,145,42,395,93,26,204,35,130,121
volksstimme_md,2022-06-02T20:04:16Z2022-06-02T09:23:28Z2022-0...,4.360336e+20,Endlich wieder Zuhause. Das war ein aufregende...,115,133,690,37,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,1,2,1,3,1,2,11,1,7,1


In [75]:
sachsenAnhaltCategorized = sachsenAnhalt
sachsenAnhaltCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/SACHSENANHALT_medienKategorisiert.csv")

## SCHLESWIG-HOLSTEIN

In [76]:
matrixSchleswigMedia = vectorizer.fit_transform(schleswig.text.astype('U'), )
words_df_SchleswigMedia = pd.DataFrame(matrixSchleswigMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [77]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    schleswig[category_name] = words_df_SchleswigMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verkehrAutomobil 
Worte: ['pkw', 'auto', 'autos', 'automobil', 'adblue', 'efuel', 'efuels', 'e-fuel', 'e-fuels', 'elektromobilität', 'emobilität', 'e-mobilität', 'emobility', 'e-mobility', 'e-auto', 'e-fahrzeug', 'e-autos', 'e-fahrzeuge', 'eauto', 'efahrzeug', 'eautos', 'efahrzeuge', 'elektroauto', 'elektroautos', 'plugin', 'ladeinfrastruktur', 'ladesäule', 'ladesäulen', 'autofahrende', 'autofahrer', 'autofahrerin', 'autofahrerinnen', 'antriebswende', 'automobilhersteller', 'verbrenner', 'verbrennungsmotor', 'verbrennerverbot', 'verkehrsverbund', 'verkehrsverbünde', 'iaa', 'autobahn', 'autobahnen', 'tempolimit', 'sonntagsfahrverbot', 'sprit', 'benzin', 'autofrei', 'autofreie', 'elektromobilitaet', 'emobilitaet'] 

Topic: verkehrFahrrad 
Worte: ['radfahrer', 'radfahrerin', 'radfahrerinnen', 'radfahren', 'fahrradfahrer', 'fahrradfahrerin', 'fahrradfahrerinnen', 'fahrrad', 'fahrradweg', 'fahrradwege', 'radweg', 'radwege', 'radfahrende', 'fahrradfreundlich', 'fahrradfreundliche', 'r

In [78]:
schleswig.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
shz_de            54
LN_Online          8
kn_online          6
FTageblatt         2
Flensborg_Avis     1
BoyensMedien       0
Name: verfassungsfeindlich, dtype: int64

In [79]:
schleswig.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
BoyensMedien,2022-10-28T15:34:52Z2022-03-18T11:28:09Z2022-0...,4.592381e+18,Das schwedische Unternehmen #Northvolt überleg...,0,5,14,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
FTageblatt,2022-10-30T10:55:34Z2022-10-29T16:57:06Z2022-1...,8.958808e+20,"Geister, Hexen, Unholde: Diese Sagen der Insel...",32,45,158,15,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,1,0,2,16,2,0,7,0,4,4
Flensborg_Avis,2022-10-31T19:01:29Z2022-10-31T18:13:42Z2022-1...,2.312402e+21,Leder: Får vi mon et pendlerspor? - https://t...,66,269,977,82,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,1,0,1,7,0,2,5,3,3,4
LN_Online,2022-10-31T17:06:20Z2022-10-31T15:45:35Z2022-1...,5.854283e+21,Ohne seine Medikamente schwebt Rolf Janßen in ...,727,584,2668,337,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,14,22,8,182,30,29,120,18,57,62
kn_online,2022-10-31T19:31:56Z2022-10-31T17:44:00Z2022-1...,6.123255e+21,Die späten Sommerferien in #SchleswigHolstein ...,3352,3414,14672,1196,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,13,23,6,121,27,43,99,23,88,91
shz_de,2022-10-31T18:30:05Z2022-10-31T18:00:19Z2022-1...,1.298624e+22,Das bestehende Personal entlasten oder als Unt...,2868,2110,8553,587,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,66,161,54,252,35,63,252,68,144,140


In [80]:
schleswigCategorized = schleswig
schleswigCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/SCHLESWIG_medienKategorisiert.csv")

## THÜRINGEN

In [81]:
matrixThüringenMedia = vectorizer.fit_transform(thüringen.text.astype('U'), )
words_df_ThüringenMedia = pd.DataFrame(matrixThüringenMedia.toarray(), columns = vectorizer.get_feature_names_out())

In [82]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    thüringen[category_name] = words_df_ThüringenMedia[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verkehrÖpnv 
Worte: ['öpnv', 'bus', 'busse', 'bahn', 'db', 'bahnhof', 'bahnhöfe', 'zug', 'züge', 'schiene', 'schienennetz', 'neuneuroticket', 'neuneurotickets', '9euroticket', '9-euro-ticket', '9€ticket', '9eurotickets', '9-euro-tickets', '9€tickets', 'deutschlandticket', 'deutschlandtickets', 'anbindung', 'öpnv-nutzer', 'öpnv-nutzerin', 'öpnv-nutzende', 'bahnfahrende', 'zugfahrende', 'nahverkehr', 'deutschlandtakt', 'deutschland-takt', 'verspätung', 'verspätungen', 'zugausfall', 'zugausfälle', 'schienenersatzverkehr', 'deutschebahn', 'oepnv', 'oepnv-nutzer', 'oepnv-nutzerin', 'oepnv-nutzende'] 

Topic: verteidigungspolitik 
Worte: ['zeitenwende', 'modernisieren', 'modernisierung', 'verteidigungspolitik', 'verteidigungsministerin', 'verteidigungsminister', 'verteidigungsministerium', 'sondervermögen', 'bundeswehr', 'truppe', 'heer', 'lambrecht', 'pistorius', 'verteidigungsfähig', 'verteidigungsfähigkeit', 'landesverteidigung'] 

Topic: zukunftEntwicklung 
Worte: ['zukunft', 'for

In [83]:
thüringen.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
OTZonline     22
TAOnline      20
TLZnews       19
freieswort     1
Name: verfassungsfeindlich, dtype: int64

In [84]:
thüringen.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
OTZonline,2022-10-31T23:01:37Z2022-10-31T22:40:06Z2022-1...,7.983540e+21,Personeller Umbruch beim FC Carl Zeiss Jena: S...,427,343,1321,101,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,11,29,22,118,9,21,74,10,48,34
TAOnline,2022-10-31T22:22:23Z2022-10-31T20:30:35Z2022-1...,8.315158e+21,"Legend of Cross in Mühlberg: Sonderschicht, So...",1569,6304,12684,639,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,22,26,20,195,11,21,75,10,43,38
TLZnews,2022-10-31T22:48:26Z2022-10-31T22:12:50Z2022-1...,8.538090e+21,"Legend of Cross in Mühlberg: Sonderschicht, So...",740,365,1015,203,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,17,25,19,146,12,16,77,11,47,40
freieswort,2022-10-26T21:59:59Z2022-10-21T21:37:37Z2022-0...,3.050110e+19,"Wie Gregor Gysi, der Entertainer in der Krise,...",16,7,23,10,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0.0,...,0,0,1,1,1,0,0,0,0,0


In [85]:
thüringenCategorized = thüringen
thüringenCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/THÜRINGEN_medienKategorisiert.csv")

# Bag of Words-Modelle für restliche Medien

D. h. für überregionale Zeitungen, den öffentlich-rechtlichen Rundfunk und alternative/digitale Medien.

In [86]:
örr = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/örr_demojized.csv",
                           index_col=0)
örr = örr.drop(columns='...1')

überregionale = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/überregio_demojized.csv",
                            index_col=0)
überregionale = überregionale.drop(columns='...1')

digitale = pd.read_csv("../../methods/representative-speech/regionalisierung/demojized/digitale_demojized.csv",
                       index_col=0)
digitale = digitale.drop(columns='...1')


## ÖRR

In [87]:
matrixÖRR = vectorizer.fit_transform(örr.text.astype('U'), )
words_df_ÖRR = pd.DataFrame(matrixÖRR.toarray(), columns = vectorizer.get_feature_names_out())

In [88]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    örr[category_name] = words_df_ÖRR[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: ukraineBewaffnung 
Worte: ['waffen', 'waffenlieferungen', 'panzer', 'kampfpanzer', 'schützenpanzer', 'gepard', 'marder', 'fuchs', 'leopard', 'munition', 'militärhilfe', 'panzerhaubitzen'] 

Topic: ukraineReaktion 
Worte: ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine', 'mariupol', 'butscha', 'saporischschja', 'cherson', 'besatzer', 'aggressor', 'völkerrechtswidrig', 'völkerrechtswidriger', 'völkerrechtswidrigen', 'völkerrechtswidrigem', 'überfall'] 

Topic: verfassungsfeindlich 
Worte: ['verschwörungstheorien', 'verschwörungstheorie', 'verschwörungstheoretikern', 'verschwörungstheoretiker', 'verschwörungstheoretikerin', 'verschwörungstheoretikerinnen', 'telegram', 'reichsbürgern', 'reichsbürger', 'reichsbürgerin', 'reichsbürgerinnen', 'querdenkern', 'querdenker', 'querdenkerin', 'querdenkerinnen', 'querdenken', 'corona-protest', 'corona-proteste', 'corona-demonstration', 'corona-demonstrationen', 'coronaprotest', 'coronaproteste', 'coronademons

In [89]:
örr.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
BR24               103
MDRAktuell          81
SWRAktuellBW        50
rbb24               41
WDRaktuell          37
DeutscheWelle       33
MDR_SN              32
hessenschau         16
mdr_th              15
SWRAktuellRP        12
DLF                 11
rbb24Inforadio       9
butenunbinnen        8
NDRinfo              7
ndr                  7
MDR_SAN              7
SWRpresse            5
WDR                  5
NDRnds               5
BR_Presse            4
SRaktuell            3
rbbabendschau        2
SRKommunikation      2
NDRsh                2
hrPresse             1
mdrde                1
MDRpresse            1
dlfnova              0
dlfkultur            0
ARTEde               0
Name: verfassungsfeindlich, dtype: int64

In [90]:
örr.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
ARTEde,2022-10-31T13:28:06Z2022-10-31T09:46:48Z2022-1...,9.112967e+20,Im Zeitalter der #Selfies erscheinen uns die S...,2169,6503,73344,1051,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/jagodamarinic/status/15844...,...,0,4,0,0,0,1,3,0,5,7
BR24,2022-10-31T21:08:00Z2022-10-31T20:30:00Z2022-1...,2.711755e+22,Auf dem Rückweg von einem #Freizeitpark am Woc...,131626,80746,270054,25872,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/jannikpentz/status/1583756...,...,261,548,103,395,54,149,517,175,238,282
BR_Presse,2022-10-28T16:24:18Z2022-10-28T12:11:03Z2022-1...,3.561641e+20,Diesen Mann muss man lieben: Mit sanfter Stimm...,449,549,1880,114,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BR24/status/15849709755982...,...,0,1,4,0,0,0,0,0,5,9
DLF,2022-10-31T07:57:53Z2022-10-31T07:40:48Z2022-1...,2.512554e+21,@guillaumemarin7 @spdde Höre dir gerne das kom...,4276,3984,16785,1238,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/CIVISPreis/status/15326812...,...,18,27,11,5,0,17,6,23,30,46
DeutscheWelle,2022-10-31T15:31:00Z2022-10-31T14:02:00Z2022-1...,2.735969e+21,"Die Infrastruktur in Cottbus ist überlastet: ""...",3154,5287,12744,658,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/dwnews/status/158571344150...,...,11,61,33,3,0,0,10,9,45,26
MDRAktuell,2022-10-31T21:41:49Z2022-10-31T20:50:00Z2022-1...,2.104257e+22,Die Internationale Atomenergiebehörde untersuc...,84167,87290,347702,24436,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AufstandLastGen/status/158...,...,248,503,81,305,15,91,364,328,121,206
MDR_SAN,2022-10-31T19:58:36Z2022-10-31T13:27:24Z2022-1...,2.783265e+21,Am Reformationstag hatten in Wittenberg unter ...,2088,3608,10390,798,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/sebwilken/status/158523592...,...,5,10,7,54,7,10,59,5,31,34
MDR_SN,2022-10-31T14:27:12Z2022-10-31T13:55:39Z2022-1...,4.099693e+21,Die Hauptsaison der #Dresdner #Weißen #Flotte ...,3915,5713,13988,1261,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/SportimOsten/status/156389...,...,11,4,32,77,10,18,64,9,40,42
MDRpresse,2022-10-30T08:50:03Z2022-10-28T09:28:19Z2022-1...,7.660581e+20,Zu #Halloween in den #Harz: Der Magdeburger @P...,448,1073,3021,210,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/HarzKurier/status/15851708...,...,1,5,1,1,1,0,2,1,9,17


In [91]:
örrCategorized = örr
örrCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/ÖRR_medienKategorisiert.csv")

## DIGITALE

In [92]:
matrixDigitale = vectorizer.fit_transform(digitale.text.astype('U'), )
words_df_Digitale = pd.DataFrame(matrixDigitale.toarray(), columns = vectorizer.get_feature_names_out())

In [93]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    digitale[category_name] = words_df_Digitale[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: verkehrAutomobil 
Worte: ['pkw', 'auto', 'autos', 'automobil', 'adblue', 'efuel', 'efuels', 'e-fuel', 'e-fuels', 'elektromobilität', 'emobilität', 'e-mobilität', 'emobility', 'e-mobility', 'e-auto', 'e-fahrzeug', 'e-autos', 'e-fahrzeuge', 'eauto', 'efahrzeug', 'eautos', 'efahrzeuge', 'elektroauto', 'elektroautos', 'plugin', 'ladeinfrastruktur', 'ladesäule', 'ladesäulen', 'autofahrende', 'autofahrer', 'autofahrerin', 'autofahrerinnen', 'antriebswende', 'automobilhersteller', 'verbrenner', 'verbrennungsmotor', 'verbrennerverbot', 'verkehrsverbund', 'verkehrsverbünde', 'iaa', 'autobahn', 'autobahnen', 'tempolimit', 'sonntagsfahrverbot', 'sprit', 'benzin', 'autofrei', 'autofreie', 'elektromobilitaet', 'emobilitaet'] 

Topic: verkehrFahrrad 
Worte: ['radfahrer', 'radfahrerin', 'radfahrerinnen', 'radfahren', 'fahrradfahrer', 'fahrradfahrerin', 'fahrradfahrerinnen', 'fahrrad', 'fahrradweg', 'fahrradwege', 'radweg', 'radwege', 'radfahrende', 'fahrradfreundlich', 'fahrradfreundliche', 'r

In [94]:
digitale.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
Volksverpetzer     685
COMPACTMagazin     116
reitschuster        56
TAG24               47
Achgut_com          46
jreichelt           45
WEBDENews           39
svz_de              35
correctiv_org       35
derfreitag          35
telepolis_news      29
TichysEinblick      27
tonline             22
OvertonMagazin      18
HalleSpektrum       18
Weltwoche           17
NNNonline           17
krautreporter       13
Verfassungsblog     12
krassundkonkret     10
ruhrbarone           9
NachDenkSeiten       9
apolut_net           9
cicero_online        9
nordbayern           8
uebermedien          8
inFranken            6
infosperber          4
talkimhangar7        4
comeon_de            3
a_watch              3
pressenza_ger        2
stv_klartext         2
JungNaiv             2
NuoVisoTV            2
mucbook              1
LRM_dietalkshow      1
langemannmedien      0
lobbycontrol         0
ServusTV_News        0
acTVismMunich        0
Hintergrund_de       0
theeuropean          0
GMX   

In [95]:
digitale.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
Achgut_com,2022-10-31T17:10:05Z2022-10-31T13:50:31Z2022-1...,2.522169e+21,#zweigeschlechter #Biologie #fakten(Pieter Cle...,7410,27653,109272,1637,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AliCologne/status/15867885...,...,13,17,46,23,1,2,12,10,24,63
COMPACTMagazin,2022-10-31T20:52:31Z2022-10-31T17:53:17Z2022-1...,3.932345e+21,Wenn es um den besonderen #Schutz fürs #herz g...,5629,18300,57490,1500,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/maxotte_says/status/149645...,...,35,70,116,10,0,3,9,9,24,17
GMX,2022-10-24T14:28:11Z2022-10-21T08:46:06Z2022-1...,6.250115e+19,Viele große Dateien per E-Mail im #Anhang vers...,23,39,77,9,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/DeutschePostDHL/status/156...,...,0,0,0,0,0,0,0,0,1,3
HalleSpektrum,2022-10-31T19:42:16Z2022-10-31T15:40:15Z2022-1...,3.682287e+21,Im Walde steht ein Pilz Wer hat sie auf seinen...,79,177,538,55,0,0.0,0,...,1,15,18,23,25,12,52,6,77,68
Hintergrund_de,2022-10-31T07:37:30Z2022-10-31T07:28:23Z2022-1...,7.934324e+18,Kinder und Jugendliche sollen frühzeitig mit d...,0,2,1,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,0,0,0,0,0,0,0,1,1,1
JungNaiv,2022-10-26T17:41:51Z2022-10-26T14:15:49Z2022-1...,1.472613e+20,Here we go: https://t.co/g1CJI22RUORegierungst...,443,618,3878,95,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/TiloJung/status/1584588474...,...,0,2,2,0,0,1,1,1,0,0
LRM_dietalkshow,2022-10-24T09:19:46Z2022-10-24T06:16:00Z2022-1...,6.556213e+20,Best of #LRM mit @mfleischhacker1\nWie #korrup...,1491,2764,11110,440,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/LRM_dietalkshow/status/157...,...,3,42,1,0,0,0,0,0,11,0
NNNonline,2022-10-31T19:00:08Z2022-10-30T14:30:12Z2022-1...,5.387091e+21,Die Rostocker #Eisbademeisters eröffnen die Ei...,307,263,1916,65,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,9,47,17,87,22,20,86,11,59,72
NachDenkSeiten,2022-10-31T13:01:08Z2022-10-31T11:04:19Z2022-1...,2.134005e+21,"Politik ohne Gott – Oder: Erst der Bürger, dan...",3930,13498,39975,1198,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/FWarweg/status/15668233164...,...,9,18,9,2,0,1,4,10,16,7


In [96]:
digitaleCategorized = digitale
digitaleCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/DIGITALE_medienKategorisiert.csv")

## ÜBERREGIONALE

In [97]:
matrixÜberregionale = vectorizer.fit_transform(überregionale.text.astype('U'), )
words_df_Überregionale = pd.DataFrame(matrixÜberregionale.toarray(), columns = vectorizer.get_feature_names_out())

In [98]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    überregionale[category_name] = words_df_Überregionale[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang', 'impfzentrum', 'impfzentren', 'ungeimpft', 'ungeimpfte', 'ungeimpfter', 'ungeimpften', 'impfgegnern', 'impfung', 'impfungen', '2g', '2gplus', '2g+', 'omicron', 'vierteimpfung'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'virologen', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion', 'sieben-tage-inzidenz', '7-tage-inzidenz', 'virologen', 'virologin', 'virologinnen', 'alpha', 'omikron', 'delta'] 

Topic: covidMaßnahmen 

Topic: sozialesWohnen 
Worte: ['wohnungsbau', 'wohnen', 'bauen', 'mieter', 'bauen', 'mieterinnen', 'mieterin', 'miete', 'mietendeckel', 'mietenstopp', 'sozialwohnung', 'sozialwohnungen'] 

Topic: ukraineAllgemein 
Worte: ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia', 'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj', 'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato', 'söldner', 'söldnergruppe', 'prigoschin', 'prigozhin', 'lawrow', 'peskow'] 

Topic: ukraineBewaffnung 
Worte: ['waffen', 'waffenlieferungen', 'panzer', 'kampfpanzer', 'schützenpanzer', 'gepard', 'marder', 'fuchs', 'leopard', 'munition', 'militärhilfe', 'panzerhaubitzen'] 

Topic: ukraineReaktion 
Worte: ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine', 'mariupol', 'butscha', 'saporischschja', 'cherson', 'besatzer', 'aggressor', 'völkerrechtswidrig', 'völkerrechtswidriger', 'völkerrechtswidrige

In [99]:
überregionale.groupby('user').verfassungsfeindlich.sum().sort_values(ascending=False)

user
RND_de            220
derspiegel        164
faznet            122
zeitonline        114
sternde           102
welt              100
ntvde              96
tazgezwitscher     89
SZ                 87
SZ_TopNews         86
tagesschau         84
taz_news           67
fr                 60
ZDFheute           52
NZZ                42
BILD               36
phoenix_de         33
focusonline        32
DIEZEIT            31
jungewelt          23
NZZde              13
dpa                 8
tagesthemen         7
ntv_EIL             7
SPIEGEL_Top         6
WELTAMSONNTAG       2
ARD_Presseclub      1
ardmoma             1
SPIEGEL_EIL         1
reuters_de          0
heutejournal        0
tagesschau24        0
tagesschau_eil      0
FOCUS_TopNews       0
BILD_News           0
FAZ_Eil             0
Name: verfassungsfeindlich, dtype: int64

In [100]:
überregionale.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,ukraineBewaffnung,ukraineReaktion,verfassungsfeindlich,verkehrAutomobil,verkehrFahrrad,verkehrPolitik,verkehrÖpnv,verteidigungspolitik,zukunftEntwicklung,zukunftVersprechen
user,,,,,,,,,,,,,,,,,,,,,
ARD_Presseclub,2022-10-30T16:20:00Z2022-10-30T16:10:00Z2022-1...,1.724157e+21,Den #presseclub gibt es auch als Podcast! hea...,22764,12712,56918,2560,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/ThePioneerDe/status/158417...,...,49,58,1,10,0,1,2,9,32,18
BILD,2022-10-31T23:04:47Z2022-10-31T23:04:46Z2022-1...,1.302323e+22,Millionen bekommen Gas-Geld erst 2023 - „Die A...,735846,225408,1072290,63617,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/Reporter_Flash/status/1587...,...,263,310,36,81,6,18,54,127,54,31
BILD_News,2022-06-05T10:58:42Z2022-01-28T09:16:59Z,3.020395e+18,@luigi_balia Hi @luigi_balia. We would like to...,2,0,5,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
DIEZEIT,2022-10-31T23:38:54Z2022-10-31T22:23:21Z2022-1...,8.102468e+21,Vor zehn Jahren kaufte #Katar einen Fußballclu...,15389,8190,40297,2478,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/MeiStueck/status/153967771...,...,57,82,31,60,9,14,42,39,131,109
FAZ_Eil,2022-10-31T10:05:10Z2022-10-30T23:06:29Z2022-1...,4.026057e+20,"Die Inflation im Euroraum steigt auf 10,7 Proz...",234,630,1365,98,0,0.0,0,...,2,2,0,2,0,1,2,2,1,1
FOCUS_TopNews,2022-10-17T12:45:02Z2022-10-14T10:45:02Z2022-1...,2.169727e+19,https://t.co/lLDF50Ay06https://t.co/lLDF50ioLY...,35,3,27,1,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/cyberport_de/status/158082...,...,0,0,0,0,0,0,0,0,0,0
NZZ,2022-10-31T21:05:00Z2022-10-31T20:18:00Z2022-1...,1.582497e+22,Die amerikanisch-deutsche Schriftstellerin Isa...,23752,29608,75436,5890,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/NZZ/status/158229441262480...,...,72,269,42,84,2,26,72,36,216,153
NZZde,2022-10-31T17:09:08Z2022-10-31T14:57:00Z2022-1...,2.794179e+21,Exil-Iraner wurden am Wochenende vor der irani...,4188,9614,33908,1326,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/NZZde/status/1572265932164...,...,34,28,13,25,4,9,30,36,24,29
RND_de,2022-10-31T17:41:21Z2022-10-31T17:15:00Z2022-1...,3.897349e+22,Der letzte große deutsche Warenhauskonzern #Ga...,97321,90397,395292,24903,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/ImreGrimm/status/152557498...,...,540,1146,220,417,16,123,466,482,344,359


In [101]:
überregionaleCategorized = überregionale
überregionaleCategorized.to_csv("../../methods/representative-speech/mediendaten/medienZugeordnet/THÜRINGEN_medienKategorisiert.csv")